In [2]:
import os
import pickle
import numpy as np
from IPython.display import clear_output
import matplotlib
import matplotlib.pyplot as plt
#import config InlineBackend.figure_format = 'svg'
import tensorflow as tf
# silencing tensorflow warnings
import logging
logging.getLogger('tensorflow').setLevel(logging.FATAL)
from datetime import datetime
tf.__version__ # printint out tensorflow version used
import stable_baselines3
import sb3_contrib
from stable_baselines3.common.callbacks import BaseCallback
# from stable_baselines3.results_plotter import load_results, ts2xy
from stable_baselines3.common.results_plotter import load_results, ts2xy
import optuna
# from stable_baselines3.bench import Monitor
from stable_baselines3.common.monitor import Monitor
#from stable_baselines3.common.policies import MlpPolicy
from stable_baselines3.common import results_plotter
from sb3_contrib.common.maskable.evaluation import evaluate_policy
from sb3_contrib import MaskablePPO
from stable_baselines3.common.vec_env import DummyVecEnv
#stable_baselines.__version__ # printing out stable_baselines version used
import gym
import pickle
import cProfile 
import pandas as pd
import ast
font = { 'family' : 'sans-serif',
                'weight' : 'normal',
                'size'   : 11}
matplotlib.rc('font', **font)

In [172]:
current_directory = os.getcwd()
# exp_id = "2022-02-09v2112frfixedrdcont"
# exp_id = "2022-02-09v42frfixedrdcont"
# exp_id = "2022-02-09v42frfixeddtagR3cont"
# exp_id = "2022-02-09v42tfbdtagR3"
# exp_id = "2022-02-09v2112tfbnsfR3"
# exp_id = "2022-02-09v22frnomask"
# exp_id_h = "2022-02-08v42frfixedrd"
# exp_id = "2022-02-08v42frfixeddtagR3"
# exp_id = "2022-02-06v421frnomask"
# exp_id = "2022-02-06v2113frnomask"
# exp_id = "2022-02-04v2112frR3cont"
# exp_id = "2022-02-03v42frR3"
# exp_id = "2022-02-03v27frR2"
exp_id = "2022-02-16v44dtag"
eval_name = "_no_tfb_test"
# eval_name = "_tfb_2500"
# eval_name = ""
eval_dir = "/Users/joshnevin/RL_FOCSLab/tmp/RWAFOCS-ppo/"+exp_id+"/_core_0/eval_results"+eval_name
# eval_dir_h = "/Users/joshnevin/RL_FOCSLab/tmp/RWAFOCS-ppo/"+exp_id_h+"/_core_0/eval_results"+eval_name

In [180]:
def get_info(eval_dir, model):
    df = pd.read_csv(eval_dir+"/evaluation_"+model+".monitor.csv", skiprows=1)
    eval_eps = df.index.values
    rewards = df['r'].to_numpy()
    sim_time = df['t'].to_numpy()
    services_accepted_ep = df['episode_services_accepted'].to_numpy()
    services_processed_ep = df['episode_services_processed'].to_numpy()
    services_accepted = df['services_accepted'].to_numpy()
    services_processed = df['services_processed'].to_numpy()
    throughput = df['throughput']/1e12
    service_dist = df['service_distribution']
    service_dist = service_dist[0]
    service_dist = ast.literal_eval(service_dist)
    service_dist = np.array(service_dist).reshape(22,100)  # NOTE! HAVE HARD CODED NUMBER OF EDGES AND CHANNELS!
    #cum_services_accepted_ep = df['episode_cum_services_accepted']
    return rewards, eval_eps, sim_time, services_accepted_ep, services_processed_ep, services_accepted, services_processed, throughput, service_dist
def get_cum_results(eval_dir, model):
    df = pd.read_csv(eval_dir+"/evaluation_"+model+".monitor.csv", skiprows=1)
    cum_services_accepted = df['episode_cum_services_accepted'].to_numpy()
    cum_services_processed = df['episode_cum_services_processed'].to_numpy()
    return cum_services_accepted, cum_services_processed

In [181]:
tfb_test = pickle.load(open(eval_dir+"/tfb_test.pkl",'rb'))
tfb_test

False

In [182]:
rewards_rl, eval_eps_rl, sim_time_rl, services_accepted_ep_rl, services_processed_ep_rl, services_accepted_rl, services_processed_rl, throughput_rl, service_dist_rl = get_info(eval_dir, 'rl')
# rewards_kspff, eval_eps_kspff, sim_time_kspff, services_accepted_ep_kspff, services_processed_ep_kspff, services_accepted_kspff, services_processed_kspff, throughput_kspff = get_info(eval_dir_h, 'kspff')
# rewards_ffksp, eval_eps_ffksp, sim_time_ffksp, services_accepted_ep_ffksp, services_processed_ep_ffksp, services_accepted_ffksp, services_processed_ffksp, throughput_ffksp = get_info(eval_dir_h, 'ffksp')
# rewards_camu, eval_eps_camu, sim_time_camu, services_accepted_ep_camu, services_processed_ep_camu, services_accepted_camu, services_processed_camu, throughput_camu = get_info(eval_dir_h, 'camu')
# rewards_kspmu, eval_eps_kspmu, sim_time_kspmu, services_accepted_ep_kspmu, services_processed_ep_kspmu, services_accepted_kspmu, services_processed_kspmu, throughput_kspmu = get_info(eval_dir_h, 'kspmu')

In [188]:
service_dist_rl[0]

array([1, 2, 2, 2, 1, 3, 2, 4, 2, 1, 1, 3, 0, 0, 2, 1, 1, 2, 4, 5, 1, 3,
       7, 8, 0, 8, 5, 1, 0, 1, 1, 1, 0, 2, 1, 0, 0, 2, 2, 1, 7, 1, 5, 2,
       1, 0, 1, 1, 5, 0, 0, 5, 0, 1, 0, 0, 3, 1, 0, 2, 1, 0, 2, 5, 0, 1,
       3, 0, 1, 8, 5, 1, 0, 5, 6, 1, 3, 3, 1, 1, 1, 1, 1, 1, 2, 1, 2, 4,
       1, 1, 2, 0, 4, 2, 2, 0, 6, 0, 4, 4])

In [162]:
# cum_services_accepted_rl, cum_services_processed_rl = get_cum_results(eval_dir, 'rl')
# test = ast.literal_eval(cum_services_accepted_rl[0])
# test2 = ast.literal_eval(cum_services_processed_rl[0])

In [163]:
mean_rl = np.mean(services_accepted_ep_rl)
freq_den = 25

In [164]:
print("Mean:",np.mean(services_accepted_ep_rl))
print("Median:",np.median(services_accepted_ep_rl))
print("Std:",np.std(services_accepted_ep_rl))

Mean: 3761.88
Median: 3763.5
Std: 14.271846411729634


In [ ]:
print("RL:",np.mean(services_accepted_ep_rl))
print("kSP-FF:",np.mean(services_accepted_ep_kspff))
print("FF-kSP:",np.mean(services_accepted_ep_ffksp))
print("CA-MU:",np.mean(services_accepted_ep_camu))
print("kSP-MU:",np.mean(services_accepted_ep_kspmu))

In [ ]:
print("RL:",np.median(services_accepted_ep_rl))
print("kSP-FF:",np.median(services_accepted_ep_kspff))
print("FF-kSP:",np.median(services_accepted_ep_ffksp))
print("CA-MU:",np.median(services_accepted_ep_camu))
print("kSP-MU:",np.median(services_accepted_ep_kspmu))

In [ ]:
print("RL:",np.std(services_accepted_ep_rl))
print("kSP-FF:",np.std(services_accepted_ep_kspff))
print("FF-kSP:",np.std(services_accepted_ep_ffksp))
print("CA-MU:",np.std(services_accepted_ep_camu))
print("kSP-MU:",np.std(services_accepted_ep_kspmu))

In [ ]:
print("RL:",np.mean(rewards_rl))
print("kSP-FF:",np.mean(rewards_kspff))
print("FF-kSP:",np.mean(rewards_ffksp))
print("CA-MU:",np.mean(rewards_camu))
print("kSP-MU:",np.mean(rewards_kspmu))

In [ ]:
np.mean(services_accepted_ep_rl) - np.mean(services_accepted_ffksp)

In [ ]:
f, ax = plt.subplots()
dict_plt = {'kSP-FF':services_accepted_ep_kspff,'kSP-MU':services_accepted_ep_kspmu,'FF-kSP':services_accepted_ep_ffksp, 'RL':services_accepted_ep_rl}
#dict_plt = {'FF-kSP':services_accepted_ep_ffksp, 'RL':services_accepted_ep_rl}
plt.boxplot(dict_plt.values(),showfliers=False,showmeans=True)
ax.set_xticklabels(dict_plt.keys())
plt.ylabel("Services accepted")
# ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
f.patch.set_facecolor('w')
# plt.ylim([2285, 2305])
plt.grid()
plt.savefig('figures/servicesaccepted'+exp_id+eval_name+'.jpeg', dpi=300,bbox_inches='tight')
plt.show()

Plotting for multiple RL agents:

In [ ]:
# exp_id_1 = "2022-02-03v42frR3"  # old top best res
# exp_id_2 = "2022-02-04v2112frR3cont" # old top with v2112 bug best res

# exp_id_1 = "2022-02-08v42frfixedrdR2"  # best res with rounded NSFNET and v2112 fixed tfb = false training
# exp_id_2 = "2022-02-09v2112frfixedrdcont"

# exp_id_4 = "2022-02-09v2112tfbnsf"


exp_id_1 = "2022-02-09v42tfbnsfR2"  # best res with rounded NSFNET and v2112 fixed tfb = True training
exp_id_h = "2022-02-08v42frfixedrd"  # NSFNET

# exp_id_1 = "2022-02-09v42frfixeddtagcont"  # best res with DTAG and v2112 fixed tfb = false training
# exp_id_2 = "2022-02-08v2112frfixeddtagR2"
# # exp_id_2 = "2022-02-09v2112tfbdtagR3"


# exp_id_1 = "2022-02-09v42tfbdtagR3"  # best res with DTAG and v2112 fixed tfb = True training
# exp_id_h = "2022-02-08v42frfixeddtagR3" # DTAG

# exp_id_3 = "2022-02-09v42frfixeddtagcont"  # best res with DTAG and v2112 fixed tfb = false training
# exp_id_4 = "2022-02-08v2112frfixeddtagR2"

eval_name_plot = "_tfb_2500"
eval_dir_1 = "/Users/joshnevin/RL_FOCSLab/tmp/RWAFOCS-ppo/"+exp_id_1+"/_core_0/eval_results"+eval_name_plot
# eval_dir_2 = "/Users/joshnevin/RL_FOCSLab/tmp/RWAFOCS-ppo/"+exp_id_2+"/_core_0/eval_results"+eval_name_plot
# eval_dir_3 = "/Users/joshnevin/RL_FOCSLab/tmp/RWAFOCS-ppo/"+exp_id_3+"/_core_0/eval_results"+eval_name_plot
# eval_dir_4 = "/Users/joshnevin/RL_FOCSLab/tmp/RWAFOCS-ppo/"+exp_id_4+"/_core_0/eval_results"+eval_name_plot
eval_dir_h = "/Users/joshnevin/RL_FOCSLab/tmp/RWAFOCS-ppo/"+exp_id_h+"/_core_0/eval_results"+eval_name_plot

In [ ]:
rewards_rl_1, eval_eps_rl_1, sim_time_rl_1, services_accepted_ep_rl_1, services_processed_ep_rl_1, services_accepted_rl_1, services_processed_rl_1, throughput_rl_1 = get_info(eval_dir_1, 'rl')
# rewards_rl_2, eval_eps_rl_2, sim_time_rl_2, services_accepted_ep_rl_2, services_processed_ep_rl_2, services_accepted_rl_2, services_processed_rl_2, throughput_rl_2 = get_info(eval_dir_2, 'rl')
# rewards_rl_3, eval_eps_rl_3, sim_time_rl_3, services_accepted_ep_rl_3, services_processed_ep_rl_3, services_accepted_rl_3, services_processed_rl_3, throughput_rl_3 = get_info(eval_dir_3, 'rl')
# rewards_rl_4, eval_eps_rl_4, sim_time_rl_4, services_accepted_ep_rl_4, services_processed_ep_rl_4, services_accepted_rl_4, services_processed_rl_4, throughput_rl_4 = get_info(eval_dir_4, 'rl')

# rewards_rl_3, eval_eps_rl_3, sim_time_rl_3, services_accepted_ep_rl_3, services_processed_ep_rl_3, services_accepted_rl_3, services_processed_rl_3, throughput_rl_3 = get_info(eval_dir_3, 'rl')
rewards_kspff, eval_eps_kspff, sim_time_kspff, services_accepted_ep_kspff, services_processed_ep_kspff, services_accepted_kspff, services_processed_kspff, throughput_kspff = get_info(eval_dir_h, 'kspff')
rewards_ffksp, eval_eps_ffksp, sim_time_ffksp, services_accepted_ep_ffksp, services_processed_ep_ffksp, services_accepted_ffksp, services_processed_ffksp, throughput_ffksp = get_info(eval_dir_h, 'ffksp')
rewards_camu, eval_eps_camu, sim_time_camu, services_accepted_ep_camu, services_processed_ep_camu, services_accepted_camu, services_processed_camu, throughput_camu = get_info(eval_dir_h, 'camu')
rewards_kspmu, eval_eps_kspmu, sim_time_kspmu, services_accepted_ep_kspmu, services_processed_ep_kspmu, services_accepted_kspmu, services_processed_kspmu, throughput_kspmu = get_info(eval_dir_h, 'kspmu')

In [ ]:
topology = 'nsfnet'
f, ax = plt.subplots()
bp = dict_plt = {'kSP-FF':services_accepted_ep_kspff,'kSP-MU':services_accepted_ep_kspmu,'FF-kSP':services_accepted_ep_ffksp,
            'RL':services_accepted_ep_rl_1}
bp = plt.boxplot(dict_plt.values(),showfliers=False,showmeans=True)
ax.set_xticklabels(dict_plt.keys())
plt.ylabel("Services accepted")
# ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
f.patch.set_facecolor('w')
# plt.legend([bp["boxes"][0]], ['A'], loc='upper right')
plt.grid()
#plt.savefig('figures/servicesaccepted_bestresults'+topology+eval_name_plot+'modelA.png', dpi=500,bbox_inches='tight')
plt.show()

In [ ]:
topology = 'dtag'
f, ax = plt.subplots()
bp = dict_plt = {'kSP-FF':services_accepted_ep_kspff,'kSP-MU':services_accepted_ep_kspmu,'FF-kSP':services_accepted_ep_ffksp,
            'RL':services_accepted_ep_rl_1, 'RL-TFB':services_accepted_ep_rl_2}
bp = plt.boxplot(dict_plt.values(),showfliers=False,showmeans=True)
ax.set_xticklabels(dict_plt.keys())
plt.ylabel("Services accepted")
# ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
f.patch.set_facecolor('w')
# plt.legend([bp["boxes"][0]], ['A'], loc='upper right')
plt.grid()
plt.savefig('figures/servicesaccepted_bestresults'+topology+eval_name_plot+'besttraintypes.png', dpi=500,bbox_inches='tight')
plt.show()

In [ ]:
topology = 'dtag'
f, ax = plt.subplots()
bp = dict_plt = {'kSP-FF':services_accepted_ep_kspff,'kSP-MU':services_accepted_ep_kspmu,'FF-kSP':services_accepted_ep_ffksp,
            'A':services_accepted_ep_rl_1, 'B':services_accepted_ep_rl_2,
                'A TFB':services_accepted_ep_rl_3, 'B TFB':services_accepted_ep_rl_4}
bp = plt.boxplot(dict_plt.values(),showfliers=False,showmeans=True)
ax.set_xticklabels(dict_plt.keys())
plt.ylabel("Services accepted")
# ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
f.patch.set_facecolor('w')
# plt.legend([bp["boxes"][0]], ['A'], loc='upper right')
plt.grid()
#plt.savefig('figures/servicesaccepted_bestresults'+topology+eval_name+'bothtraintypes.png', dpi=500,bbox_inches='tight')
plt.show()

In [ ]:
medians = [item.get_ydata()[0] for item in bp['medians']]
means = [item.get_ydata()[0] for item in bp['means']]
print(f'Medians: {medians}\n'
      f'Means:   {means}')

In [ ]:
minimums = [round(item.get_ydata()[0], 1) for item in bp['caps']][::2]
maximums = [round(item.get_ydata()[0], 1) for item in bp['caps']][1::2]
print(f'Minimums: {minimums}\n'
      f'Maximums: {maximums}')

In [ ]:
q1 = [round(min(item.get_ydata()), 1) for item in bp['boxes']]
q3 = [round(max(item.get_ydata()), 1) for item in bp['boxes']]
# print(f'Q1: {q1}\n'
#       f'Q3: {q3}')
IQR = [q3[i] - q1[i] for i in range(len(q1))]
print(IQR)

In [ ]:
print("RL A:",np.std(services_accepted_ep_rl_1))
print("RL B:",np.std(services_accepted_ep_rl_2))
print("RL A TFB:",np.std(services_accepted_ep_rl_3))
print("RL B TFB:",np.std(services_accepted_ep_rl_4))

In [ ]:
print("kSP-FF:",np.std(services_accepted_ep_kspff))
print("FF-kSP:",np.std(services_accepted_ep_ffksp))
print("CA-MU:",np.std(services_accepted_ep_camu))
print("kSP-MU:",np.std(services_accepted_ep_kspmu))

In [ ]:
print("RL:",np.mean(services_accepted_ep_rl) + np.std(services_accepted_ep_rl))
print("kSP-FF:",np.mean(services_accepted_ep_kspff) +np.std(services_accepted_ep_kspff))
print("FF-kSP:",np.mean(services_accepted_ep_ffksp) +np.std(services_accepted_ep_ffksp))
print("CA-MU:",np.mean(services_accepted_ep_camu) +np.std(services_accepted_ep_camu))
print("kSP-MU:",np.mean(services_accepted_ep_kspmu) +np.std(services_accepted_ep_kspmu))

In [ ]:
100*(2159-2151)/2159

In [ ]:
100*(2156-2151)/2156

In [ ]:
((341-21)/2156)*100

In [ ]:
((2158-2099)/2099)*100

In [ ]:
((2151-2092)*25)/1000

In [ ]:
2151-2099

In [ ]:
((2151-2099)*25)/1000

In [ ]:
((3416-3256)*25)/1000

In [ ]:
((3416-3256)/3256)*100

In [ ]:
3416-3256

In [ ]:
((2158-2156)/2156)*100

In [ ]:
y_max = 25
f, ax = plt.subplots()
plt.hist(services_accepted_ep_kspff, label='kSP-FF')
plt.hist(services_accepted_ep_camu, label='CA-MU')
plt.hist(services_accepted_ep_kspmu, label='kSP-MU')
plt.hist(services_accepted_ep_ffksp, label='FF-kSP')
plt.hist(services_accepted_ep_rl, label='RL')
plt.vlines(np.mean(services_accepted_ep_rl), 0, y_max, color='k',linestyles='--', label='RL mean')
plt.vlines(np.mean(services_accepted_ep_ffksp), 0, y_max, color='k',linestyles='-.', label='FF-kSP mean')
plt.xlabel("Services accepted")
plt.ylabel("Frequency density")
plt.ylim([0,y_max])
plt.grid()
ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
f.patch.set_facecolor('w')
plt.legend(ncol = 2,loc='upper center', prop = {'size':10})
# plt.savefig('figures/servicesacceptedv2111f25Gbid.jpeg', dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(eval_eps_kspff, throughput_kspff, '+', label='kSP-FF')
plt.plot(eval_eps_ffksp, throughput_ffksp, '+', label='FF-kSP')
plt.plot(eval_eps_camu, throughput_camu, '+', label='CA-MU')
plt.plot(eval_eps_kspmu, throughput_kspmu, '+', label='kSP-MU')
plt.plot(eval_eps_rl, throughput_rl, '+', label='RL')
plt.xlabel("Evaluation episodes")
plt.ylabel("Throughput")
plt.legend()
plt.show()

In [ ]:
plt.plot(eval_eps_kspff, services_accepted_ep_kspff, 'rx', label='kSP-FF')
# plt.plot(eval_eps_ffksp, services_accepted_ep_ffksp, 'rP', label='FF-kSP')
# plt.plot(eval_eps_camu, services_accepted_ep_camu, 'k.', label='CA-MU')
# plt.plot(eval_eps_kspmu, services_accepted_ep_kspmu, 'm*', label='kSP-MU')
plt.plot(eval_eps_rl, services_accepted_ep_rl, 'b-', label='RL')
plt.xlabel("Evaluation episodes")
plt.ylabel("Services accepted")
plt.legend()
plt.show()

Timing results

In [ ]:
rl_time = pickle.load(open("/Users/joshnevin/RL_FOCSLab/eval_timing/"+exp_id+"_RL.pkl", 'rb'))
kspff_time = pickle.load(open("/Users/joshnevin/RL_FOCSLab/eval_timing/"+exp_id+"_kspff.pkl", 'rb'))
ffksp_time = pickle.load(open("/Users/joshnevin/RL_FOCSLab/eval_timing/"+exp_id+"_ffksp.pkl", 'rb'))
camu_time = pickle.load(open("/Users/joshnevin/RL_FOCSLab/eval_timing/"+exp_id+"_camu.pkl", 'rb'))
kspmu_time = pickle.load(open("/Users/joshnevin/RL_FOCSLab/eval_timing/"+exp_id+"_kspmu.pkl", 'rb'))

In [ ]:
print(rl_time)
print(kspff_time)
print(ffksp_time)
print(camu_time)
print(kspmu_time)